## Preprocessing

In [12]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load the dataset
url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
df = pd.read_csv(url)

# Display the first few rows to ensure the data is loaded correctly
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [13]:
# Identify target (y) and features (X)
X = df.drop(columns=['EIN', 'NAME', 'IS_SUCCESSFUL'])  # Drop identification columns
y = df['IS_SUCCESSFUL']  # Target variable

# Determine the number of unique values in each column
df.nunique()

,0
EIN,34299
NAME,19568
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2


In [14]:
# Combine rare categorical values in the 'APPLICATION_TYPE' column
application_counts = X['APPLICATION_TYPE'].value_counts()

# Set a threshold to combine rare values
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace rare application types with 'Other'
for app in application_types_to_replace:
    X['APPLICATION_TYPE'] = X['APPLICATION_TYPE'].replace(app, 'Other')

# Check that replacement was successful
X['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [15]:
# Encode categorical variables using pd.get_dummies()
X = pd.get_dummies(X)

# Check the new structure of the DataFrame
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,False,False,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [16]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()

# Fit the scaler on the training data, then apply it to the training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
# Build the neural network model
model = tf.keras.models.Sequential()

# Add the input layer and two hidden layers
model.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Add the output layer (binary classification)
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 80)                  │           8,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,181 (43.68 KB)

 Trainable params: 11,181 (43.68 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6896 - loss: 0.5979 - val_accuracy: 0.7378 - val_loss: 0.5592
Epoch 2/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7286 - loss: 0.5537 - val_accuracy: 0.7363 - val_loss: 0.5551
Epoch 3/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7302 - loss: 0.5480 - val_accuracy: 0.7374 - val_loss: 0.5556
Epoch 4/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7311 - loss: 0.5527 - val_accuracy: 0.7382 - val_loss: 0.5525
Epoch 5/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7362 - loss: 0.5408 - val_accuracy: 0.7382 - val_loss: 0.5526
Epoch 6/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7378 - loss: 0.5403 - val_accuracy: 0.7321 - val_loss: 0.5523
Epoch 7/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7311 - loss: 0.5460 - val_accuracy: 0.7371 - val_loss: 0.5547
Epoch 8/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7306 - loss: 0.5437 - val_accuracy: 0.

In [19]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7282 - loss: 0.5528
Test Loss: 0.5570021271705627, Test Accuracy: 0.7290087342262268
